In [ ]:
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from openai import AzureOpenAI

# Initialize Azure Form Recognizer client
form_recognizer_endpoint = "https://spjdocumentintelligence.cognitiveservices.azure.com/"
form_recognizer_key = "88c92c39f1794aafb2c84d2d7826f9d6"
form_recognizer_client = DocumentAnalysisClient(endpoint=form_recognizer_endpoint, credential=AzureKeyCredential(form_recognizer_key))

# Initialize OpenAI client
openai_api_key = "7d8a1f9f0ace4c19ac8fb19c82827521"
openai_endpoint = "https://spjopenai9902916927.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2023-07-01-preview"
openai_client = AzureOpenAI(api_key=openai_api_key, api_version="2023-07-01-preview", azure_endpoint=openai_endpoint)

def extract_dataframe_from_pdf(file_path):
    # Read file content
    with open(file_path, "rb") as file:
        file_content = file.read()

    # Analyze document
    poller = form_recognizer_client.begin_analyze_document("prebuilt-document", file_content)
    result = poller.result()

    # Extract table data
    table_data = []
    for table in result.tables:
        table_dict = {}
        for cell in table.cells:
            if cell.row_index not in table_dict:
                table_dict[cell.row_index] = {}
            table_dict[cell.row_index][cell.column_index] = cell.content.strip()
        table_data.append(table_dict)

    # Convert table data to DataFrame
    dfs = []
    for table in table_data:
        headers = sorted(table[0].keys())  # Use sorted headers as column names
        df = pd.DataFrame.from_dict(table, orient='index', columns=headers)
        dfs.append(df)

    # Return the first DataFrame (assuming there's only one table in the PDF)
    if dfs:
        return df.rename(columns=df.iloc[0]).drop(df.index[0])
    else:
        return None

def suggest_nutrient_intake(openai_client, df):
    # Convert DataFrame to text
    text = df.to_string(index=False, header=True)
    prompt = "You are a medical blood test report analyzer and nutrition expert. See for the test values if they are higher or lower than the reference range. But dont give give this as the ouput. your output should be the nutrients that we should intake for the observations from the analysis. You should only provide the information from the national institute of health us information website.Can you make the format in a better way. easily readable and has some designs. Only provide the nurtition information for those results which are either high or low."
    # Convert DataFrame to text
    text = df.to_string(index=False, header=True)
    response = openai_client.chat.completions.create(
        model="gpt-4-medical",
        temperature=0.3,
        top_p=0.95,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        max_tokens=800,
        messages=[
            {"role": "system", "content": "Assistant is a medical blood test report analyzer and nutrition expert."},
            {"role": "user", "content": f"{prompt}"},
            {"role": "assistant", "content": f"{text}"}
        ]
    )
    return response.choices[0].message.content.strip()

# Example usage
file_path = r"C:\Users\hp\Desktop\Robonomics\nutrify\blood report samples.pdf"
df = extract_dataframe_from_pdf(file_path)
if df is not None:

    nutrient = suggest_nutrient_intake(openai_client, df)
    print(nutrient)
else:
    print("No DataFrame extracted from PDF.")
    
